In [1]:
import os
import fnmatch
import numpy as np
import re
import seaborn as sns

import matplotlib.pyplot as plt
import pandas as pd
import math

from matplotlib import animation
from matplotlib.animation import FFMpegWriter

import numpy as np
from scipy.stats import skew

In [2]:
def _is_coord_line(parts):
    if len(parts) != 3:
        return False
    try:
        int(parts[0])
        float(parts[1])
        float(parts[2])
        return True
    except ValueError:
        return False

        
def _is_demand_line(parts):
    if len(parts) != 2:
        return False
    try:
        int(parts[0]); int(parts[1])
        return True
    except ValueError:
        return False


class Instance:
    def __init__(self, filename):
        self.filename = filename
        
        # the name of the instance 
        self.name = None
        
        # the variables below can be read directly from the 
        self.optimum = None # the optimum value
        self.num_of_vehicles = None
        self.dimension = None
        self.num_of_customers = None
        self.num_of_stations = None # the number of recharging stations 
        self.capacity = None # the maximum capacity of vehicle
        self.battery_capacity = None # the battery capacity of EV
        self.energy_consumption = None # the energy consumption rate
        self.edge_weight_format = None
        self.node_coordinates = {} # dictionary {'0': [145, 215], ...}
        self.demands = {} # {'0': 0, '1':1100}
        self.station_list = [] # ['22', '23', '24'] 
        self.depot_index = None # 1
        
        # the variables below can be gotten after processing
        self.actual_problem_size = None # Total number of customers, charging stations and depot
        self.distance_matrix = None
        
        self.read_problem()
            
    def read_problem(self):   
        self.name = os.path.splitext(self.filename.split('/')[-1])[0]
        with open(self.filename, 'rt', encoding='utf-8', newline='') as f:
            line = f.readline().strip()
            while line:
                if line.startswith('OPTIMAL_VALUE:'):
                    # 兼容：OPTIMAL_VALUE: 167575 (Best Known)
                    # 以及旧版：OPTIMAL_VALUE: 167575
                    m = re.search(r'OPTIMAL_VALUE:\s*([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)', line)
                    if m:
                        self.optimum = float(m.group(1))
                    else:
                        raise Exception(f"Invalid OPTIMAL_VALUE line: {line}")
                elif line.startswith('VEHICLES:'):
                    self.num_of_vehicles = int(line.split()[-1])
                elif line.startswith('DIMENSION:'):
                    self.dimension = int(line.split()[-1])
                elif line.startswith('STATIONS:'):
                    self.num_of_stations = int(line.split()[-1])
                elif line.startswith('CAPACITY:'):
                    self.capacity = int(line.split()[-1])
                elif line.startswith('ENERGY_CAPACITY:'):
                    self.battery_capacity = int(line.split()[-1])       
                elif line.startswith('ENERGY_CONSUMPTION:'):
                    self.energy_consumption = float(line.split()[-1])
                elif line.startswith('EDGE_WEIGHT_FORMAT:') or line.startswith('EDGE_WEIGHT_TYPE:'):
                    self.edge_weight_format = line.split()[-1]  
                elif line.startswith('NODE_COORD_SECTION'):
                    # 读到不能再读坐标行为止
                    line = f.readline().strip()
                    while line:
                        parts = line.split()
                        if _is_coord_line(parts):
                            self.node_coordinates[str(int(parts[0]) - 1)] = [float(parts[1]), float(parts[2])]
                            line = f.readline().strip()
                        else:
                            # 遇到下一段的标题行（如 DEMAND_SECTION）或其它非坐标行
                            break
                    continue # 让外层 while 用当前的 line 继续判断下一段
                elif line.startswith('DEMAND_SECTION'):
                    demand_cnt = 0  # 统计 demand 行数

                    # 同理：读到不能再读需求行为止
                    line = f.readline().strip()
                    while line:
                        parts = line.split()
                        if _is_demand_line(parts):
                            self.demands[str(int(parts[0]) - 1)] = int(parts[1])
                            demand_cnt += 1
                            line = f.readline().strip()
                        else:
                            break
                    
                    # 对新老实例统一适用：customer 数 = demand 行数 - 1（去掉 depot）
                    if demand_cnt > 0:
                        self.num_of_customers = demand_cnt - 1
                    else:
                        raise Exception("Empty DEMAND_SECTION")
        
                    continue           
                elif line.startswith('STATIONS_COORD_SECTION'):
                    for i in range(self.num_of_stations):
                        self.station_list.append(str(int(f.readline().split()[0]) - 1))    
                elif line.startswith('DEPOT_SECTION'):
                    self.depot_index = str(int(f.readline().strip()) - 1)
                line = f.readline().strip() 
        
        # make some process the data
        self.actual_problem_size = self.num_of_customers + self.num_of_stations + 1

        
        if self.edge_weight_format == 'EUC_2D':
            node_coords = np.array(list(self.node_coordinates.values()))
            self.distance_matrix = np.sqrt(np.sum((node_coords[:,np.newaxis] - node_coords)**2, axis=2))            
        else:
            raise ValueError("Unsupported edge weight format: {}".format(self.edge_weight_format))

    def get_distance(self, node_1, node_2):
        '''Get the Euclidean distance between two nodes.

        Args:
            node_1: a index of a node
            node_2: a index of a node

        Returns:
            a real number denoting the distance  
        '''
        return self.distance_matrix[int(node_1)][int(node_2)]

In [3]:
BASE_DIR = os.path.abspath(os.path.dirname(os.path.dirname('.')))
data_dir = os.path.join(BASE_DIR, 'data')
result_dir = os.path.join(BASE_DIR, 'data_results')

data_dir

'/data/home/exx866/Instance-Aware-Algorithm-Configuration/data'

In [6]:
filenames = fnmatch.filter(os.listdir(data_dir), '*.evrp')
# 提取 -nXXX- 中的数字并排序
def extract_n_value(filename):
    match = re.search(r'-n(\d+)-', filename)
    return int(match.group(1)) if match else float('inf')

sorted_filenames = sorted(filenames, key=extract_n_value)
# sorted_filenames

In [7]:
filename = os.path.join(data_dir, "E-n22-k4.evrp")
inst = Instance(filename)
# inst.__dict__

## Graph/Topology features for E-CVRP instances

In [8]:
filename = os.path.join(data_dir, "X-n830-k171-s11.evrp")
inst = Instance(filename)
# inst.__dict__

In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Graph/Topology features for EVRP instances using NetworkX.

Assumptions (matching your Instance reader):
- instance.node_coordinates: dict[str, [x, y]] for ALL nodes (customers + stations), indexed from 0
- instance.demands: dict[str, demand] for customers only
- instance.station_list: list[str] of station node indices (0-based as strings)
- instance.depot_index: str of depot node index (0-based)
- instance.actual_problem_size = dimension + num_of_stations
- instance.distance_matrix: NxN numpy array of Euclidean distances

We build a sparse graph (kNN) to compute topology features efficiently.
We also compute MST on the kNN graph (or fallback to full metric if needed).
"""

from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, Any, List, Tuple, Optional

import numpy as np
import networkx as nx


# -----------------------------
# Utilities
# -----------------------------

def _safe_float(x: float) -> float:
    # ensure JSON-serializable floats (avoid numpy types)
    return float(x)


def _clip_int(x: int, lo: int, hi: int) -> int:
    return max(lo, min(hi, x))


def _as_int_ids(instance) -> List[int]:
    # node ids are 0..N-1
    return list(range(int(instance.actual_problem_size)))


def _coord_array(instance) -> np.ndarray:
    # node_coordinates is dict[str, [x,y]]
    N = int(instance.actual_problem_size)
    coords = np.zeros((N, 2), dtype=float)
    for i in range(N):
        coords[i, :] = instance.node_coordinates[str(i)]
    return coords


def _node_types(instance) -> Dict[int, str]:
    """
    Return node type: 'depot', 'customer', 'station'
    """
    depot = int(instance.depot_index)
    stations = set(int(s) for s in instance.station_list)
    types: Dict[int, str] = {}
    N = int(instance.actual_problem_size)
    for i in range(N):
        if i == depot:
            types[i] = "depot"
        elif i in stations:
            types[i] = "station"
        else:
            types[i] = "customer"
    return types


# -----------------------------
# Graph construction
# -----------------------------

def build_knn_graph(instance, k: int = 10, mutual: bool = True) -> nx.Graph:
    """
    Build a weighted kNN graph using Euclidean distances from instance.distance_matrix.

    Parameters
    ----------
    k : number of nearest neighbors per node (clipped to [1, N-1])
    mutual : if True, keep an undirected edge (i,j) only if i is in kNN(j) AND j in kNN(i)
             if False, use the union (i in kNN(j) OR j in kNN(i))

    Returns
    -------
    G : networkx.Graph with:
        - nodes: 0..N-1
        - edge attribute 'weight' = Euclidean distance
    """
    N = int(instance.actual_problem_size)
    if N < 2:
        raise ValueError("Instance must have at least 2 nodes to build a graph.")
    k = _clip_int(int(k), 1, N - 1)

    D = instance.distance_matrix  # NxN numpy array
    # For each i, find k nearest j != i
    knn = []
    for i in range(N):
        row = D[i].copy()
        row[i] = np.inf
        # nbrs = np.argpartition(row, k)[:k]
        nbrs = np.argpartition(row, k - 1)[:k]
        nbrs = nbrs[np.argsort(row[nbrs])]  # sort by distance
        knn.append(set(int(j) for j in nbrs))

    G = nx.Graph()
    G.add_nodes_from(range(N))

    if mutual:
        for i in range(N):
            for j in knn[i]:
                if i in knn[j]:
                    w = float(D[i][j])
                    if i != j:
                        G.add_edge(i, j, weight=w)
    else:
        for i in range(N):
            for j in knn[i]:
                w = float(D[i][j])
                if i != j:
                    G.add_edge(i, j, weight=w)

    # ensure connectedness if possible: if disconnected, we will keep it but
    # downstream features will compute on giant component where needed.
    return G


def build_complete_graph(instance) -> nx.Graph:
    """
    Build a complete weighted graph (O(N^2) edges). Only use for small instances.
    """
    N = int(instance.actual_problem_size)
    D = instance.distance_matrix
    G = nx.Graph()
    G.add_nodes_from(range(N))
    for i in range(N):
        for j in range(i + 1, N):
            G.add_edge(i, j, weight=float(D[i][j]))
    return G


def ensure_connected_graph(G: nx.Graph) -> Tuple[nx.Graph, bool]:
    """
    Return (Gc, was_connected)
    - If G connected: return (G, True)
    - Else: return (largest connected component induced subgraph, False)
    """
    if nx.is_connected(G):
        return G, True
    comps = sorted(nx.connected_components(G), key=len, reverse=True)
    giant = comps[0]
    return G.subgraph(giant).copy(), False


# -----------------------------
# Feature extraction
# -----------------------------

def compute_graph_features(
    instance,
    *,
    k: int = 10,
    mutual_knn: bool = True,
    use_complete_graph_if_small: bool = False,
    complete_graph_threshold: int = 10,
) -> Dict[str, Any]:
    """
    Compute topology/graph features for a given instance.

    Strategy:
    - Build kNN graph (or complete graph if N <= threshold and enabled).
    - Use giant component for path-based statistics if graph disconnected.
    - Compute MST weight (over the chosen graph). If graph disconnected, MST is for giant component.
    - Compute selected centralities and clustering stats.
    - Compute spectral features of the normalized Laplacian on the giant component.

    Returns
    -------
    features : dict[str, Any]
    """
    N = int(instance.actual_problem_size)
    depot = int(instance.depot_index)
    node_types = _node_types(instance)

    # Choose graph
    if use_complete_graph_if_small and N <= complete_graph_threshold:
        G = build_complete_graph(instance)
        graph_kind = "complete"
    else:
        G = build_knn_graph(instance, k=k, mutual=mutual_knn)
        graph_kind = f"knn(k={_clip_int(k,1,max(1,N-1))}, mutual={mutual_knn})"

    # Connectivity
    Gc, is_conn = ensure_connected_graph(G)
    n_gc = Gc.number_of_nodes()
    m_gc = Gc.number_of_edges()

    # Basic stats
    degrees = np.array([d for _, d in G.degree()], dtype=float)
    degrees_gc = np.array([d for _, d in Gc.degree()], dtype=float)

    # Weighted edges
    weights = np.array([edata["weight"] for _, _, edata in G.edges(data=True)], dtype=float) if G.number_of_edges() else np.array([])
    weights_gc = np.array([edata["weight"] for _, _, edata in Gc.edges(data=True)], dtype=float) if m_gc else np.array([])

    # MST (on Gc)
    mst = nx.minimum_spanning_tree(Gc, weight="weight")
    mst_weight = sum(edata["weight"] for _, _, edata in mst.edges(data=True)) if mst.number_of_edges() else 0.0
    mst_degrees = np.array([d for _, d in mst.degree()], dtype=float) if mst.number_of_nodes() else np.array([0.0])

    # Clustering (unweighted clustering; weighted clustering exists but is slower)
    clustering_gc = nx.clustering(Gc)
    clustering_vals = np.array(list(clustering_gc.values()), dtype=float) if clustering_gc else np.array([0.0])

    # Path-based features (on giant component only)
    # For kNN graphs, shortest paths exist if connected; if not, we use giant component.
    # Weighted shortest path length:
    try:
        apl_w = nx.average_shortest_path_length(Gc, weight="weight") if n_gc > 1 else 0.0
    except Exception:
        apl_w = float("nan")

    # Unweighted diameter / eccentricity
    try:
        diameter = nx.diameter(Gc) if n_gc > 1 else 0
    except Exception:
        diameter = None

    # Depot centrality in Gc (if depot outside giant component, set NaN)
    depot_in_gc = depot in Gc.nodes
    if depot_in_gc and n_gc > 1:
        # betweenness can be expensive; on big graphs consider approximation
        betw = nx.betweenness_centrality(Gc, k=None, normalized=True, weight="weight")
        clos = nx.closeness_centrality(Gc, distance="weight")
        dep_betw = float(betw.get(depot, float("nan")))
        dep_clos = float(clos.get(depot, float("nan")))
    else:
        dep_betw = float("nan")
        dep_clos = float("nan")

    # Degree assortativity (may be NaN for small graphs)
    try:
        assort = nx.degree_assortativity_coefficient(Gc)
    except Exception:
        assort = float("nan")

    # Spectral features (normalized Laplacian eigenvalues)
    # Use only small/medium giant component to keep it safe; for huge graphs, take k smallest eigenvalues via sparse methods (not used here).
    spectral = {}
    if n_gc <= 1500 and n_gc > 2:
        try:
            Gc_sim = nx.Graph()
            Gc_sim.add_nodes_from(Gc.nodes())
            eps = 1e-9
            for u, v, edata in Gc.edges(data=True):
                d = float(edata["weight"])
                Gc_sim.add_edge(u, v, weight=1.0/(d+eps))
            
            L = nx.normalized_laplacian_matrix(Gc_sim, weight="weight").astype(float)
            # L = nx.normalized_laplacian_matrix(Gc, weight="weight").astype(float)
            # Convert to dense for eigvalsh when size manageable
            Ld = L.toarray()
            evals = np.linalg.eigvalsh(Ld)
            # summary stats
            spectral["lap_eig_min"] = _safe_float(np.min(evals))
            spectral["lap_eig_max"] = _safe_float(np.max(evals))
            spectral["lap_eig_mean"] = _safe_float(np.mean(evals))
            spectral["lap_eig_std"] = _safe_float(np.std(evals))
            # algebraic connectivity = second smallest eigenvalue (Fiedler value)
            spectral["algebraic_connectivity"] = _safe_float(np.sort(evals)[1])
        except Exception:
            spectral["lap_eig_min"] = float("nan")
            spectral["lap_eig_max"] = float("nan")
            spectral["lap_eig_mean"] = float("nan")
            spectral["lap_eig_std"] = float("nan")
            spectral["algebraic_connectivity"] = float("nan")
    else:
        spectral["lap_eig_min"] = float("nan")
        spectral["lap_eig_max"] = float("nan")
        spectral["lap_eig_mean"] = float("nan")
        spectral["lap_eig_std"] = float("nan")
        spectral["algebraic_connectivity"] = float("nan")

    # Spatial/geometric quick extras (helpful alongside graph features)
    coords = _coord_array(instance)
    # pairwise distances summary via distance matrix (avoid O(N^2) if N huge)
    # Here we sample pairs for large N
    D = instance.distance_matrix
    if N <= 800:
        upper = D[np.triu_indices(N, k=1)]
    else:
        rng = np.random.default_rng(0)
        idx_i = rng.integers(0, N, size=20000)
        idx_j = rng.integers(0, N, size=20000)
        mask = idx_i != idx_j
        upper = D[idx_i[mask], idx_j[mask]]

    # nearest neighbor distance per node
    nn = []
    for i in range(N):
        row = D[i].copy()
        row[i] = np.inf
        nn.append(float(np.min(row)))
    nn = np.array(nn, dtype=float)

    features: Dict[str, Any] = {
        # meta
        "graph_kind": graph_kind,
        "N_nodes": N,
        "depot_in_giant_component": bool(depot_in_gc),
        "is_connected": bool(is_conn),
        "N_giant": int(n_gc),
        "M_giant": int(m_gc),

        # degree stats (whole graph)
        "deg_mean": _safe_float(np.mean(degrees)) if len(degrees) else 0.0,
        "deg_std": _safe_float(np.std(degrees)) if len(degrees) else 0.0,
        "deg_min": _safe_float(np.min(degrees)) if len(degrees) else 0.0,
        "deg_max": _safe_float(np.max(degrees)) if len(degrees) else 0.0,

        # degree stats (giant component)
        "deg_gc_mean": _safe_float(np.mean(degrees_gc)) if len(degrees_gc) else 0.0,
        "deg_gc_std": _safe_float(np.std(degrees_gc)) if len(degrees_gc) else 0.0,

        # edge weight stats (giant component)
        "edge_w_gc_mean": _safe_float(np.mean(weights_gc)) if len(weights_gc) else 0.0,
        "edge_w_gc_std": _safe_float(np.std(weights_gc)) if len(weights_gc) else 0.0,
        "edge_w_gc_min": _safe_float(np.min(weights_gc)) if len(weights_gc) else 0.0,
        "edge_w_gc_max": _safe_float(np.max(weights_gc)) if len(weights_gc) else 0.0,

        # clustering
        "clust_gc_mean": _safe_float(np.mean(clustering_vals)) if len(clustering_vals) else 0.0,
        "clust_gc_std": _safe_float(np.std(clustering_vals)) if len(clustering_vals) else 0.0,

        # MST
        "mst_weight": _safe_float(mst_weight),
        "mst_weight_per_node": _safe_float(mst_weight / max(1, n_gc - 1)),
        "mst_deg_max": _safe_float(np.max(mst_degrees)) if len(mst_degrees) else 0.0,
        "mst_deg_mean": _safe_float(np.mean(mst_degrees)) if len(mst_degrees) else 0.0,

        # shortest paths / diameter
        "avg_shortest_path_w": _safe_float(apl_w) if apl_w == apl_w else float("nan"),
        "diameter_unweighted": int(diameter) if diameter is not None else None,

        # centrality (depot)
        "depot_betweenness_w": _safe_float(dep_betw) if dep_betw == dep_betw else float("nan"),
        "depot_closeness_w": _safe_float(dep_clos) if dep_clos == dep_clos else float("nan"),

        # assortativity
        "degree_assortativity": _safe_float(assort) if assort == assort else float("nan"),

        # spectral
        **spectral,

        # geometric summaries (distance matrix based)
        "pairdist_mean": _safe_float(np.mean(upper)) if len(upper) else 0.0,
        "pairdist_std": _safe_float(np.std(upper)) if len(upper) else 0.0,
        "pairdist_cv": _safe_float(np.std(upper) / np.mean(upper)) if len(upper) and np.mean(upper) > 0 else 0.0,
        "nn_dist_mean": _safe_float(np.mean(nn)) if len(nn) else 0.0,
        "nn_dist_std": _safe_float(np.std(nn)) if len(nn) else 0.0,
        "nn_dist_cv": _safe_float(np.std(nn) / np.mean(nn)) if len(nn) and np.mean(nn) > 0 else 0.0,
    }

    # Optional: type-based stats (customers vs stations)
    types = _node_types(instance)
    customers = [i for i, t in types.items() if t == "customer"]
    stations = [i for i, t in types.items() if t == "station"]
    features["num_customers"] = len(customers)
    features["num_stations"] = len(stations)

    # average customer->nearest station distance (use distance matrix)
    if stations and customers:
        dcs = []
        for c in customers:
            dcs.append(float(np.min(D[c, stations])))
        dcs = np.array(dcs, dtype=float)
        features["cust_to_station_nn_mean"] = _safe_float(np.mean(dcs))
        features["cust_to_station_nn_std"] = _safe_float(np.std(dcs))
    else:
        features["cust_to_station_nn_mean"] = float("nan")
        features["cust_to_station_nn_std"] = float("nan")

    return features


# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    # Example (adapt to your project):
    # from your_instance_reader import Instance
    inst = Instance(filename)
    feats = compute_graph_features(inst, k=10, mutual_knn=True)
    print(feats)

    print("This module provides compute_graph_features(instance, ...).")
    print("Integrate it with your Instance class and call it per instance.")


{'graph_kind': 'knn(k=10, mutual=True)', 'N_nodes': 830, 'depot_in_giant_component': True, 'is_connected': True, 'N_giant': 830, 'M_giant': 3152, 'deg_mean': 7.595180722891566, 'deg_std': 1.878213943602178, 'deg_min': 1.0, 'deg_max': 10.0, 'deg_gc_mean': 7.595180722891566, 'deg_gc_std': 1.878213943602178, 'edge_w_gc_mean': 26.48629332152885, 'edge_w_gc_std': 18.18425846532247, 'edge_w_gc_min': 1.0, 'edge_w_gc_max': 153.5838533179839, 'clust_gc_mean': 0.5908978772231784, 'clust_gc_std': 0.1558989098719094, 'mst_weight': 14383.32667646546, 'mst_weight_per_node': 17.35021311998246, 'mst_deg_max': 4.0, 'mst_deg_mean': 1.997590361445783, 'avg_shortest_path_w': 634.8333198756574, 'diameter_unweighted': 36, 'depot_betweenness_w': 0.37188743786530537, 'depot_closeness_w': 0.0021194509994602686, 'degree_assortativity': 0.4057535389842504, 'lap_eig_min': 2.1546358657722796e-16, 'lap_eig_max': 1.7935121683620046, 'lap_eig_mean': 1.0, 'lap_eig_std': 0.42441933199584864, 'algebraic_connectivity': 0

{'graph_kind': 'complete', 'N_nodes': 830, 'depot_in_giant_component': True, 'is_connected': True, 'N_giant': 830, 'M_giant': 344035, 'deg_mean': 829.0, 'deg_std': 0.0, 'deg_min': 829.0, 'deg_max': 829.0, 'deg_gc_mean': 829.0, 'deg_gc_std': 0.0, 'edge_w_gc_mean': 532.4557328489265, 'edge_w_gc_std': 288.29913651109644, 'edge_w_gc_min': 1.0, 'edge_w_gc_max': 1339.334909572658, 'clust_gc_mean': 1.0, 'clust_gc_std': 0.0, 'mst_weight': 14335.685072470109, 'mst_weight_per_node': 17.292744357623775, 'mst_deg_max': 4.0, 'mst_deg_mean': 1.997590361445783, 'avg_shortest_path_w': 532.4557328489141, 'diameter_unweighted': 1, 'depot_betweenness_w': 0.0, 'depot_closeness_w': 0.0023443289774536745, 'degree_assortativity': nan, 'lap_eig_min': nan, 'lap_eig_max': nan, 'lap_eig_mean': nan, 'lap_eig_std': nan, 'algebraic_connectivity': nan, 'pairdist_mean': 536.291812020334, 'pairdist_std': 289.53556428487394, 'pairdist_cv': 0.5398843648090155, 'nn_dist_mean': 13.699835162424682, 'nn_dist_std': 13.23340282230281, 'nn_dist_cv': 0.9659534341404937, 'num_customers': 818, 'num_stations': 11, 'cust_to_station_nn_mean': 103.41707624109557, 'cust_to_station_nn_std': 37.31492264185843}

In [10]:
import numpy as np
from scipy.stats import skew
import pdb

def compute_demand_features(instance):
    Q = float(instance.capacity) if instance.capacity is not None else 0.0
    depot = int(instance.depot_index) if instance.depot_index is not None else None

    # 只取 customers：demands 里去掉 depot
    keys = []
    for k in instance.demands.keys():
        ik = int(k)
        if depot is not None and ik == depot:
            continue
        keys.append(ik)
    keys.sort()

    demands = np.array([float(instance.demands[str(k)]) for k in keys], dtype=float)

    feats = {}

    if demands.size == 0:
        # 没有 customer 的极端情况
        feats.update({
            "demand_mean": 0.0,
            "demand_std": 0.0,
            "demand_cv": 0.0,
            "demand_to_capacity_mean": np.nan,
            "demand_to_capacity_max": np.nan,
            "total_demand_to_capacity": np.nan,
            "demand_skewness": 0.0,
            "demand_entropy": 0.0,
            "high_demand_ratio": 0.0,
            "demand_weighted_station_dist": np.nan,
        })
        return feats

    # basic stats
    mu = float(np.mean(demands))
    sd = float(np.std(demands))
    feats["demand_mean"] = mu
    feats["demand_std"] = sd
    feats["demand_cv"] = float(sd / mu) if mu > 0 else 0.0

    # capacity tightness
    if Q > 0:
        feats["demand_to_capacity_mean"] = float(mu / Q)
        feats["demand_to_capacity_max"] = float(np.max(demands) / Q)
        feats["total_demand_to_capacity"] = float(np.sum(demands) / Q)
    else:
        feats["demand_to_capacity_mean"] = np.nan
        feats["demand_to_capacity_max"] = np.nan
        feats["total_demand_to_capacity"] = np.nan

    # distribution shape
    feats["demand_skewness"] = float(skew(demands)) if demands.size >= 3 else 0.0

    # entropy (use probabilities, not density)
    counts, _ = np.histogram(demands, bins="auto", density=False)
    counts = counts[counts > 0]
    p = counts / counts.sum()
    feats["demand_entropy"] = float(-np.sum(p * np.log(p)))

    # high demand ratio
    alpha = 0.5
    feats["high_demand_ratio"] = float(np.mean(demands > alpha * Q)) if Q > 0 else np.nan

    # demand-weighted distance to nearest station
    stations = [int(s) for s in instance.station_list] if instance.station_list is not None else []
    D = instance.distance_matrix
    if D is None or len(stations) == 0 or np.sum(demands) <= 0:
        feats["demand_weighted_station_dist"] = np.nan
    else:
        # keys 和 demands 对齐，直接 zip
        weighted = []
        for idx, d in zip(keys, demands):
            dist = float(np.min(D[idx, stations]))
            weighted.append(d * dist)
        feats["demand_weighted_station_dist"] = float(np.sum(weighted) / np.sum(demands))

    return feats


In [11]:
feats = compute_demand_features(inst)
print(feats)

{'demand_mean': 74.46699266503667, 'demand_std': 14.484694257337729, 'demand_cv': 0.194511604926655, 'demand_to_capacity_mean': 0.20800835939954376, 'demand_to_capacity_max': 0.27932960893854747, 'total_demand_to_capacity': 170.15083798882682, 'demand_skewness': 0.05309679003134673, 'demand_entropy': 2.3902902988690244, 'high_demand_ratio': 0.0, 'demand_weighted_station_dist': 103.36038564330435}


In [12]:
import os
import pandas as pd

records = []

for fname in sorted_filenames:
    path = os.path.join(data_dir, fname)
    print(f"Processing {fname} ...")

    try:
        inst = Instance(path)

        graph_feats = compute_graph_features(inst)
        demand_feats = compute_demand_features(inst)

        # 合并所有特征
        feats = {}
        feats.update(graph_feats)
        feats.update(demand_feats)

        # 加一些元信息（非常有用）
        feats["instance_name"] = inst.name
        feats["filename"] = fname
        feats["N_total"] = inst.actual_problem_size
        feats["num_customers"] = inst.num_of_customers
        feats["num_stations"] = len(inst.station_list)
        feats["num_depot"] = 1
        feats["vehicle_capacity"] = inst.capacity
        feats["battery_capacity"] = inst.battery_capacity
        feats["energy_consumption"] = inst.energy_consumption
        feats["num_vehicles"] = inst.num_of_vehicles
        feats["max_evals"] = inst.actual_problem_size * 25000
        

        records.append(feats)

    except Exception as e:
        print(f"❌ Failed on {fname}: {e}")


# 汇总为 DataFrame
df = pd.DataFrame(records)

# 想放前面的“元信息列”（用你真实的字段名）
meta_cols = [
    "instance_name",
    "filename",  # 建议保留
    "graph_kind",
    "N_total",
    "num_customers",
    "num_stations",
    "num_depot",
    "num_vehicles",
    "vehicle_capacity",
    "battery_capacity",
    "energy_consumption",
    "max_evals",
]

# 只选择 реально存在的列，避免 KeyError
meta_cols_present = [c for c in meta_cols if c in df.columns]
other_cols = [c for c in df.columns if c not in meta_cols_present]
df = df[meta_cols_present + other_cols]

# 保存为 CSV
out_csv = os.path.join(data_dir, "evrp_instance_features.csv")
df.to_csv(out_csv, index=False)

print(f"\n✅ Feature table saved to: {out_csv}")
print(df.shape)


Processing E-n22-k4.evrp ...
Processing E-n23-k3.evrp ...
Processing E-n29-k4-s7.evrp ...
Processing E-n30-k3.evrp ...
Processing E-n30-k3-s7.evrp ...
Processing E-n33-k4.evrp ...
Processing E-n35-k3-s5.evrp ...
Processing E-n37-k4-s4.evrp ...
Processing F-n49-k4-s4.evrp ...
Processing E-n51-k5.evrp ...
Processing E-n60-k5-s9.evrp ...
Processing E-n76-k7.evrp ...
Processing F-n80-k4-s8.evrp ...
Processing E-n89-k7-s13.evrp ...
Processing E-n101-k8.evrp ...
Processing M-n110-k10-s9.evrp ...
Processing E-n112-k8-s11.evrp ...
Processing M-n126-k7-s5.evrp ...
Processing F-n140-k5-s5.evrp ...
Processing X-n143-k7.evrp ...
Processing X-n147-k7-s4.evrp ...
Processing M-n163-k12-s12.evrp ...
Processing M-n212-k16-s12.evrp ...
Processing X-n214-k11.evrp ...
Processing X-n221-k11-s7.evrp ...
Processing X-n351-k40.evrp ...
Processing X-n360-k40-s9.evrp ...
Processing X-n459-k26.evrp ...
Processing X-n469-k26-s10.evrp ...
Processing X-n573-k30.evrp ...
Processing X-n577-k30-s4.evrp ...
Processing 